# Description

This notebook is to generate known good outputs (KGOs) for solving KorakianitisMixedModel (KMM) after we have added a cycle step size argument to the `advance_cycle()` function of the solver.

With different step sizes, there are slightly different solutions reached, so tests need these different KGOs to have as comparisons.

In [1]:
import numpy as np
import json

from ModularCirc.Solver import Solver

from ModularCirc.Models.KorakianitisMixedModel import KorakianitisMixedModel
from ModularCirc.Models.KorakianitisMixedModel_parameters import KorakianitisMixedModel_parameters

In [ ]:
time_setup_dict = {
            'name': 'TimeTest',
            'ncycles': 40,
            'tcycle': 1.0,
            'dt': 0.001,
            'export_min': 1
        }

# Start the loop with different cycle step size to generate JSON files for each loop:
cycle_step_sizes = [1, 3, 5, 7]

# Dictionary to store the results for all cycle step sizes
all_results = {
    "metadata": {
        "description": "Results for different cycle_step_size values",
        "cycle_step_sizes": cycle_step_sizes
    },
    "results": {}
}

# Loop through each cycle step size
for i_cycle_step_size in cycle_step_sizes:

    print(f"Running simulation with cycle step size: {i_cycle_step_size}")

    # Initializing the parameter object
    parobj = KorakianitisMixedModel_parameters()

    # Initializing the model 
    model = KorakianitisMixedModel(time_setup_dict=time_setup_dict, 
                                parobj=parobj, 
                                suppress_printing=True)

    # Initializing the solver
    solver = Solver(model=model)

    # Initializing the parameter object
    parobj = KorakianitisMixedModel_parameters()

    # Initializing the model 
    model = KorakianitisMixedModel(time_setup_dict=time_setup_dict, 
                                parobj=parobj, 
                                suppress_printing=True)

    # Initializing the solver
    solver = Solver(model=model)

    # Solver is being setup: switching off console printing and setting the solver method to "LSODA"
    solver.setup(suppress_output=True, 
                method='LSODA',
                step=i_cycle_step_size)

    # Running the model
    solver.solve()

    # Define the indexes of the equivalent to the last cycles
    tind_fin  = np.arange(start=model.time_object.n_t-model.time_object.n_c,
                        stop=model.time_object.n_t)

    # Dictionary to store the final cycle values for each component for the current step size
    final_cycle_values = {}

    # From each of the components, retrieve the volume (<V>), pressure (<P_i>) and flow (<Q_i>)
    for key, value in model.components.items():
        final_cycle_values[key] = {
            'V': value.V.values[tind_fin].mean(),
            'P_i': value.P_i.values[tind_fin].mean(),
            'Q_i': value.Q_i.values[tind_fin].mean()
        }

    # Add the results for the current cycle step size to the main dictionary
    all_results["results"][i_cycle_step_size] = final_cycle_values

# Save all results to a single JSON file in the tests/expected_outputs directory
with open('../../tests/expected_outputs/KorakianitisMixedModel_expected_output.json', 'w') as f:
    json.dump(all_results, f, indent=4)

Running simulation with cycle step size: 1
Running simulation with cycle step size: 3
Running simulation with cycle step size: 5
Running simulation with cycle step size: 7
